In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

In [2]:
batch_size = 50
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=1, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64,32,kernel_size=3,stride=1,padding=1,bias=False)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32,32,kernel_size=3,stride=1,padding=1,bias=False)
        self.bn3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(32,16,kernel_size=3,stride=1,padding=1,bias=False)
        self.bn4 = nn.BatchNorm2d(16)
        
        self.fc1 = nn.Linear(2*512,8*512)
        self.fc2 = nn.Linear(8*512,2*512)
        self.fc3 = nn.Linear(2*512,512)
        self.fc4 = nn.Linear(512, num_classes)
        
        self.shortcut = nn.Sequential(
                nn.Conv2d(3, 16, kernel_size=1, stride=1,padding=1, bias=False),
                nn.BatchNorm2d(16))


    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = F.relu(self.bn3(self.conv3(out)))
        out = F.relu(self.bn4(self.conv4(out)))
        out.add(self.shortcut(x))
        out = F.avg_pool2d(out,4)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = F.relu(self.fc3(out))
        out = F.relu(self.fc4(out))

        return out

In [18]:
model = ResNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [19]:
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader,0):
        inputs,labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
    correct = 0
    total = 0
    for images,labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted==labels).sum()
    accuracy = 100*correct/total
    print("Iteration: {}. Loss: {}. Accuracy: {}.".format(epoch, loss.item(), accuracy))

        
    

Iteration: 0. Loss: 1.2508938312530518. Accuracy: 44.
Iteration: 1. Loss: 1.160209059715271. Accuracy: 51.
Iteration: 2. Loss: 0.9418920278549194. Accuracy: 59.
Iteration: 3. Loss: 1.002698540687561. Accuracy: 61.
Iteration: 4. Loss: 0.9742993116378784. Accuracy: 62.
Iteration: 5. Loss: 1.0632413625717163. Accuracy: 63.
Iteration: 6. Loss: 0.5952156782150269. Accuracy: 70.
Iteration: 7. Loss: 0.13888199627399445. Accuracy: 70.
Iteration: 8. Loss: 0.12766362726688385. Accuracy: 70.
Iteration: 9. Loss: 0.018593082204461098. Accuracy: 71.
